In [3]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms as T
import matplotlib.pyplot as plt
from tqdm import tqdm

# Define data directories
dataset_path = "/home/lubin/Tipu12/Tipu-12/"
train_path = dataset_path + "train"
val_path = dataset_path + "val"
test_path = dataset_path + "test"

# Load the pretrained ResNet50 model
model = timm.create_model('resnet50.a1_in1k', pretrained=True)
num_features = model.get_classifier().in_features

# Modify the classifier for 12 classes
model.fc = nn.Linear(num_features, 12)

# Freeze all layers except the last layer
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

# Set model to training mode
model.train()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Define data transforms
data_config = timm.data.resolve_model_data_config(model)
train_transforms = timm.data.create_transform(**data_config, is_training=True)
val_transforms = timm.data.create_transform(**data_config, is_training=False)

# Load datasets
train_dataset = datasets.ImageFolder(root=train_path, transform=train_transforms)
val_dataset = datasets.ImageFolder(root=val_path, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Lists to store metrics
train_losses = []
val_losses = []
val_accuracies = []

# Training loop with validation
print("Training started...")
num_epochs = 100
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0

    with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}", unit='batch') as pbar:
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            pbar.update(1)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    
    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
            
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    
    epoch_val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = 100 * correct / total

    val_losses.append(epoch_val_loss)
    val_accuracies.append(val_accuracy)
    
    print(f"Epoch: {epoch}, Training Loss: {epoch_loss:.4f}, Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Plotting the loss and accuracy graphs
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(14, 5))

# Plot training and validation loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'b-', label='Training Loss')
plt.plot(epochs, val_losses, 'r-', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, val_accuracies, 'g-', label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Training started...


Epoch 1/100:   2%|▏         | 14/706 [00:17<14:48,  1.28s/batch]


KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
test_transforms = timm.data.create_transform(**data_config, is_training=False)

test_dataset = datasets.ImageFolder(root=test_path, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
        
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

test_loss = test_loss / len(test_loader.dataset)
test_accuracy = 100 * correct / total

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")